In [1]:
import cv2
import numpy as np

import os
from matplotlib import pyplot as plt

In [2]:
working_p = "./"
in_p = working_p + "in/"
out_p = working_p + "out/"

files = [f for f in os.listdir(in_p)]

In [3]:
imgs = []
for f in files:
    imgs.append((f, cv2.imread(in_p + f, cv2.IMREAD_GRAYSCALE)))

Helper functions

In [4]:
def do_fft(img):
    f = np.fft.fft2(img)
    fshift= np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    return (fshift, magnitude_spectrum)

def do_ifft(fshift):
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    return img_back

def create_file_name(file, suffix):
    return file.split(".")[0] + suffix + "." + file.split(".")[-1]

Code

In [5]:
clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(20,20))

for (file, img) in imgs:
    (fshift, magnitude_spectrum) = do_fft(img)
    
    big_r = 18
    small_r = 7

    rows, cols = img.shape
    crow,ccol = int(rows/2) , int(cols/2)
    big_r = int(rows * big_r / 100)
    small_r = int(rows * small_r / 100)
    med_r = abs(int(big_r - (big_r - small_r) / 2))
    
    # empty image
    empty_img = np.ones((rows, cols), np.uint8)
    cv2.circle(empty_img, (ccol, crow), med_r, color=0, thickness=abs(big_r - small_r))

    empty_img = 1 - empty_img
    fshift_mod = fshift * empty_img
    
    fshift_mod *= 8
# -----------------------------------------------------------------------------

    ms_mod = 20*np.log(np.abs(fshift_mod))

# -----------------------------------------------------------------------------
    img_mod = do_ifft(fshift_mod)
    img_mod = cv2.convertScaleAbs(img_mod)

    # canny
    # img_mod = cv2.Canny(img_mod, 100, 200)

    img_mod = cv2.threshold(img_mod, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # img_mod = cv2.adaptiveThreshold(img_mod, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 51, 0)

    # hist,bins = np.histogram(img_mod.flatten(),256,[0,256])
    # cdf = hist.cumsum()
    # cdf_normalized = cdf * float(hist.max()) / cdf.max()
    # plt.plot(cdf_normalized, color = 'b')
    # plt.hist(img_mod.flatten(),256,[0,256], color = 'r')
    # plt.xlim([0,256])
    # plt.legend(('cdf','histogram'), loc = 'upper left')
    # plt.title('Histogram for image: ' + file)
    # plt.show()

    # rows, cols = img_mod.shape
    # for x in range(rows):
    #     for y in range(cols):
            # img_mod[x][y] = max(0, img_mod[x][y] - 20)
            # img_mod[x][y] *= 1.2
            # img_mod[x][y] = max(0, img_mod[x][y] - 80)

    # img_mod = clahe.apply(img_mod)
    # img_mod = np.ones((rows, cols), np.uint8) * 255 - img_mod

    # filterSize =(3, 3)
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT,
    #                                filterSize)
    # img_mod = cv2.morphologyEx(img_mod, 
    #                           cv2.MORPH_BLACKHAT,
    #                           kernel)

    # img_mod = cv2.adaptiveThreshold(img_mod, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 51, 0)

# -----------------------------------------------------------------------------

    cv2.imwrite(out_p + create_file_name(file, "_1"), img)
    cv2.imwrite(out_p + create_file_name(file, "_2"), magnitude_spectrum)
    cv2.imwrite(out_p + create_file_name(file, "_3"), ms_mod)
    cv2.imwrite(out_p + create_file_name(file, "_4"), img_mod)

/tmp/ipykernel_337869/2590312387.py:25: RuntimeWarning: divide by zero encountered in log
  ms_mod = 20*np.log(np.abs(fshift_mod))
